In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as la
import pandas as pd
from __future__ import division

# Define properties

In [2]:
number_of_groups = 4
width = 154
nodes = 10

In [3]:
# Cross sections ordered from lowest energy to highest energy

# fission is actually fission * neutrons per fission
fuel_fission = np.array([
        0.009572,
        0.001193,
        0.01768,
        0.18514
    ])

fuel_transport = np.array([
        1.38741155,
        2.76065151,
        4.74833808,
        8.46740051
    ])

fuel_scattering = np.array([
        [0, 0.083004, 0,      0      ],
        [0, 0,        0.0584, 0      ],
        [0, 0,        0,      0.06453],
        [0, 0,        0,      0      ]
    ])

fuel_absorption = np.array([
        0.004946,
        0.002840,
        0.03053,
        0.1210
    ])

fuel_removal = np.array([
        0.08795,
        0.06124,
        0.09506,
        0.1210
    ])

# Not a cross section. 1 / (3 sigma_tr)
fuel_diffusion = np.array([
        2.1623,
        1.0867,
        0.6318,
        0.3543
    ])

# Fraction of fission neutrons born into each group
fuel_chi = np.array([
        1.,
        0.,
        0.,
        0.
    ])

In [4]:
fuel_removal - fuel_absorption

array([ 0.083004,  0.0584  ,  0.06453 ,  0.      ])

In [5]:
3 / fuel_diffusion

array([ 1.38741155,  2.76065151,  4.74833808,  8.46740051])

In [6]:
# Reflector properties

reflector_fission = np.array([
        0, 0, 0, 0
    ])

reflector_transport = np.array([
        0.20608,
        0.60215,
        0.56830,
        1.21110
    ])

reflector_absorption = np.array([
        0.00051,
        0.00354,
        0.01581,
        0.04637
    ])

reflector_scattering = np.array([
        [0.37045, 0.04152, 0.00001, 0.00000],
        [0.00000, 0.98285, 0.07459, 0.01371],
        [0.00000, 0.00000, 0.76110, 0.31856],
        [0.00000, 0.00000, 0.00085, 1.96607]
    ])

reflector_diffusion = 1 / (3 * reflector_transport)

In [7]:
reflector_diffusion

array([ 1.61749482,  0.55357192,  0.58654467,  0.27523188])

# Define the matrix for each group

## Fuel matrix (all groups)

In [65]:
def fuel_matrices(radius, nodes, sigma_a, sigma_f, neutrons_per_fission, num_groups, sigma_tr=None, diffusion=None):
    # Placeholder
    mats = np.zeros((nodes - 1, nodes - 1, num_groups))
    if diffusion is None:
        diffusion = 1 / (3 * sigma_tr)
        
    delta_r = radius / (nodes - 1)
    
    for group in range(num_groups):
        tmp = mats[:,:,group]
        
        fission = sigma_f[group]
        absorption = sigma_a[group]
        diff = diffusion[group]
        
        diag = np.ones(tmp.shape[0]) * (2 * diff / delta_r ** 2 + absorption)
        diag[0] = 2 * diff / delta_r ** 2 + absorption / 2

        # Define our upper diagonal elements
        upper_diag = np.zeros(tmp.shape[0] - 1)
        for i in range(upper_diag.shape[0]):
            upper_diag[i] = -diff / delta_r ** 2 * (1 + 1 / (2 * i - 1))
        upper_diag[0] = -2 * diff / delta_r ** 2
        
        # Define our lower diagonal elements
        lower_diag = np.zeros(tmp.shape[0] - 1)
        for i in range(lower_diag.shape[0]):
            lower_diag[i] = -diff / delta_r ** 2 * (1 - 1 / (2 * i - 1))
        lower_diag[tmp.shape[0] - 2] = -diff / delta_r + diff / (2 * radius)
        
        tmp = tmp + np.diagflat(diag) + np.diagflat(upper_diag, 1) + np.diagflat(lower_diag, -1)
        
        mats[:,:,group] = tmp
    
    # for mat_index in range(mats.shape[2]):
    #     print mats[:,:,mat_index]
    
    return mats

In [71]:
fuel_matrices(5, 6, fuel_removal, fuel_fission, 2.43 * fuel_chi, number_of_groups, diffusion=fuel_diffusion)[:,:,0]

array([[ 4.368575  , -4.3246    ,  0.        ,  0.        ,  0.        ],
       [-4.3246    ,  4.41255   , -4.3246    ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  4.41255   , -2.88306667,  0.        ],
       [ 0.        ,  0.        , -1.44153333,  4.41255   , -2.59476   ],
       [ 0.        ,  0.        ,  0.        , -1.94607   ,  4.41255   ]])

## Reflector matrix (all groups)

In [ ]:
def reflector_matrices(
    width,
    nodes,
    sigma_a,
    num_groups,
    sigma_tr=None,
    diffusion=None,
    fuel_tr=None,
    fuel_diffusion=None,
    fuel_width=None
):
    # Placeholder
    mats = np.zeros(nodes - 1, nodes - 1, num_groups)
    
    delta_r = width / (nodes - 1)
    
    if diffusion is None:
        diffusion = 1 / (3 * sigma_tr)
    if fuel_diffusion is None:
        fuel_diffusion = 1 / (3 * fuel_tr)
    if fuel_width is None:
        raise Exception("You must provide a fuel width")
    
    for group in num_groups:
        tmp = mats[:,:,group]
        
        absorption = sigma_a[group]
        diff = diffusion[group]
        fd = fuel_diffusion[group]
        
        diag = np.ones(tmp.shape[0]) * (2 * diff / delta_r ** 2 + absorption)
        diag[0] = fd / delta_r - fd / (2 * fuel_width)  
        

# Start calculating flux for each group

In [ ]:
fluxes = []
for g in number_of_groups:
    flux = np.ones(nodes - 1)
    fluxes.append(flux)

In [ ]:
def power(group, min_error=0.00001, max_iterations=1e5):
    flux = fluxes[group - 1]
    k_eff = 1.0
    flux_old = np.ones(nodes-1) * 100.
    k_eff_old = 1.0
    
    flux_error = 1.0
    k_eff_error = 1.0
    
    iterations = 0
    
    source = np.ones_like(flux) * nusig
    source[0] = source[0] * 0.5
    
    mat = matrix(nodes, width, absorption, transfer)
    
    while ((flux_error > min_error) or (k_eff_error > min_error)) and (iterations < max_iterations):
        # iterate generations
        iterations += 1
        
        flux = la.solve(mat, np.multiply(flux, source) / k_eff)
        
        # solve for eigenvalue
        k_eff = k_eff_old * la.norm(np.multiply(flux, source)) / la.norm(np.multiply(flux_old, source))
        
        flux = flux / la.norm(flux)
        
        flux_error = np.amax(abs((np.multiply(flux, source) - np.multiply(flux_old, source)) / np.multiply(flux, source)))
        k_eff_error = (abs((k_eff - k_eff_old) / k_eff))
        
        k_eff_old = k_eff
        flux_old = flux
        
        if (iterations % 10000 == 0):
            print flux_error
    
    return flux, k_eff, flux_error, k_eff_error, iterations

In [10]:
a = np.array(range(25)).reshape((5,5))
z = np.zeros(5)

In [11]:
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [12]:
z

array([ 0.,  0.,  0.,  0.,  0.])

In [23]:
for i in range(5):
    a = np.vstack((a, z))

In [24]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

In [30]:
a.shape[0]

10

In [40]:
np.hstack((a, np.zeros(a.shape[0]).reshape((a.shape[0], 1))))

array([[  0.,   1.,   2.,   3.,   4.,   0.],
       [  5.,   6.,   7.,   8.,   9.,   0.],
       [ 10.,  11.,  12.,  13.,  14.,   0.],
       [ 15.,  16.,  17.,  18.,  19.,   0.],
       [ 20.,  21.,  22.,  23.,  24.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.]])

In [39]:
np.zeros(a.shape[0])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [37]:
[np.zeros(a.shape[0])]

[array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]

In [41]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.]])

In [46]:
a = np.vsplit(a, 2)[0]

In [47]:
a

array([[  0.,   1.,   2.,   3.,   4.],
       [  5.,   6.,   7.,   8.,   9.],
       [ 10.,  11.,  12.,  13.,  14.],
       [ 15.,  16.,  17.,  18.,  19.],
       [ 20.,  21.,  22.,  23.,  24.]])

In [52]:
z = np.array(range(36)).reshape((6,6))

In [53]:
z

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35]])

In [54]:
np.zeros((a.shape[0], z.shape[1])).shape

(5, 6)

In [55]:
np.zeros((z.shape[0], a.shape[1])).shape

(6, 5)

In [57]:
t_left = np.vstack((a, np.zeros((z.shape[0], a.shape[1]))))
t_right = np.vstack((np.zeros((a.shape[0], z.shape[1])), z))
m = np.hstack((t_left, t_right))
m

array([[  0.,   1.,   2.,   3.,   4.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  5.,   6.,   7.,   8.,   9.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 10.,  11.,  12.,  13.,  14.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 15.,  16.,  17.,  18.,  19.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 20.,  21.,  22.,  23.,  24.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   1.,   2.,   3.,   4.,   5.],
       [  0.,   0.,   0.,   0.,   0.,   6.,   7.,   8.,   9.,  10.,  11.],
       [  0.,   0.,   0.,   0.,   0.,  12.,  13.,  14.,  15.,  16.,  17.],
       [  0.,   0.,   0.,   0.,   0.,  18.,  19.,  20.,  21.,  22.,  23.],
       [  0.,   0.,   0.,   0.,   0.,  24.,  25.,  26.,  27.,  28.,  29.],
       [  0.,   0.,   0.,   0.,   0.,  30.,  31.,  32.,  33.,  34.,  35.]])